In [111]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

In [156]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1 
min_mem_size=6 
run_time=100
classification=False
scale=False
max_models=20   
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0

In [157]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

0


In [158]:
#Funtion for meta data
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,
                  balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data




In [159]:
#Function to convert dictionary into json file
def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
    
#Function for random alphabet   
def alphabet(n):
    alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
    str=''
    r=len(alpha)-1   
    while len(str)<n:
        i=random.randint(0,r)
        str+=alpha[i]   
        return str

def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return

def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id  
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()     
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val
    return d    

In [160]:
all_variables=None

In [161]:
data_path='C:/Users/pkash/Downloads/adult.csv'

In [162]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id)

R


In [163]:
##For logs
logfile=run_id+'_autoh2o_log.zip'
logs_path=os.path.join(run_dir,'logs')
print(logs_path,' ',logfile)

C:\Users\pkash\Y\e\Q\R\logs   R_autoh2o_log.zip


In [164]:
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O cluster uptime:,4 mins 10 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,"14 days, 17 hours and 2 minutes"
H2O cluster name:,H2O_from_python_pkash_feeqih
H2O cluster total nodes:,1
H2O cluster free memory:,1.428 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [165]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)

{'start_time': 1555360951.0231917, 'target': None, 'server_path': 'C:\\Users\\pkash\\Y\\e\\Q', 'data_path': 'C:/Users/pkash/Downloads/adult.csv', 'test_path': None, 'max_models': 20, 'run_time': 100, 'run_id': 'R', 'scale': False, 'classification': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1555360951.0231917, 'execution_time': 0.0, 'run_path': 'C:\\Users\\pkash\\Y\\e\\Q\\R', 'nthreads': 1, 'min_mem_size': 0, 'analysis': 0}


In [166]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [167]:
df.head()

age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [168]:

df.describe()

Rows:48842
Cols:15




,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
type,int,enum,int,enum,int,enum,enum,enum,enum,enum,int,int,int,enum,enum
mins,17.0,,12285.0,,1.0,,,,,,0.0,0.0,1.0,,
mean,38.643585438761704,,189664.13459727284,,10.078088530363212,,,,,,1079.0676262233324,87.50231358257237,40.42238237582409,,
maxs,90.0,,1490400.0,,16.0,,,,,,99999.0,4356.0,99.0,,
sigma,13.710509934443555,,105604.02542315728,,2.570972755592256,,,,,,7452.019057655393,403.00455212435907,12.391444024252307,,
zeros,0,,0,,0,,,,,,44807,46560,0,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [169]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
    target=df.columns[14]   
y = target
y

'income'

In [170]:
x = list(df.columns)
x.remove(y)
x
# Add independent variables

meta_data['x']=x 
# impute missing values

_=impute_missing_values(df,x, scale)


In [171]:
# # impute missing values

# _ = df[ints].impute(method='median')

# if scale:
    
#     df[ints] = df[ints].scale()

In [183]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [184]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok

In [186]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [187]:
classification=True
if classification:
    print(df[y].levels())

[['<=50K', '>50K']]


In [188]:
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    

In [189]:
allV=get_variables_types(df)
allV

{'age': 'int',
 'workclass': 'enum',
 'fnlwgt': 'int',
 'education': 'enum',
 'educational-num': 'int',
 'marital-status': 'enum',
 'occupation': 'enum',
 'relationship': 'enum',
 'race': 'enum',
 'gender': 'enum',
 'capital-gain': 'int',
 'capital-loss': 'int',
 'hours-per-week': 'int',
 'native-country': 'enum',
 'income': 'enum'}

In [190]:
meta_data['variables']=allV

In [191]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [192]:
model_start_time = time.time()

In [193]:
aml.train(x=x,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [194]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [195]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [196]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190415_164309,0.928763,0.277231,0.167004,0.296432,0.087872
1,GBM_2_AutoML_20190415_164528,0.928615,0.277539,0.173465,0.296479,0.087900
2,StackedEnsemble_AllModels_AutoML_20190415_164309,0.928563,0.294461,0.176996,0.300726,0.090436
3,GBM_1_AutoML_20190415_164528,0.928524,0.277594,0.167987,0.296648,0.088000
4,GBM_1_AutoML_20190415_164309,0.928404,0.278311,0.167706,0.296534,0.087932
5,GBM_3_AutoML_20190415_164528,0.928247,0.278928,0.168976,0.296887,0.088142
6,StackedEnsemble_BestOfFamily_AutoML_20190415_164309,0.927830,0.295549,0.169280,0.301231,0.090740
7,GBM_3_AutoML_20190415_164309,0.925747,0.288488,0.173977,0.299740,0.089844
8,GBM_4_AutoML_20190415_164528,0.919004,0.329096,0.173070,0.315762,0.099706
9,GBM_4_AutoML_20190415_164309,0.918170,0.363044,0.180648,0.330738,0.109387


In [197]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [198]:
mod_best._id

'GBM_2_AutoML_20190415_164309'

In [199]:
# Get stacked ensemble  
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
se=get_stacked_ensemble(model_set)

In [200]:
print(se)

StackedEnsemble_BestOfFamily_AutoML_20190415_164309


In [201]:
if se is not None:
  mod_best=h2o.get_model(se)

In [202]:

dir(mod_best)

['F0point5',
 'F1',
 'F2',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_run_time',
 '_start_time',
 '_train',
 '_verify_training_frame_params',
 '_xval_keys',
 'accuracy',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'base_models',
 'biases',
 'blending_frame',
 'catoffsets',
 'coef',
 'coef_norm',
 'confusion_matrix',

In [203]:
mod_best._id

'StackedEnsemble_BestOfFamily_AutoML_20190415_164309'

In [204]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics(o, train, valid, xval)>

In [205]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [206]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190415_164309
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0736600356819248
RMSE: 0.2714038240001876
LogLoss: 0.24454375975837792
Null degrees of freedom: 48841
Residual degrees of freedom: 48838
Null deviance: 53750.68147229072
Residual deviance: 23888.01262823739
AIC: 23896.01262823739
AUC: 0.9559127280066483
pr_auc: 0.840407477798703
Gini: 0.9118254560132966
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2970655986423669: 


,<=50K,>50K,Error,Rate
<=50K,33880.0,3275.0,0.0881,(3275.0/37155.0)
>50K,2148.0,9539.0,0.1838,(2148.0/11687.0)
Total,36028.0,12814.0,0.111,(5423.0/48842.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2970656,0.7786621,245.0
max f2,0.1200239,0.8519735,324.0
max f0point5,0.6736415,0.8150871,120.0
max accuracy,0.4193484,0.8953565,199.0
max precision,0.9633192,1.0,0.0
max recall,0.0419706,1.0,386.0
max specificity,0.9633192,1.0,0.0
max absolute_mcc,0.4193484,0.7066471,199.0
max min_per_class_accuracy,0.2056475,0.8752460,282.0
max mean_per_class_accuracy,0.1332593,0.8801255,317.0


Gains/Lift Table: Avg response rate: 23.93 %, avg score: 24.10 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2853130,0.7284661,250.0
max f2,0.0899907,0.8078645,341.0
max f0point5,0.7434509,0.7641827,94.0
max accuracy,0.5102429,0.8746775,168.0
max precision,0.9640764,1.0,0.0
max recall,0.0355351,1.0,399.0
max specificity,0.9640764,1.0,0.0
max absolute_mcc,0.4080050,0.6432872,202.0
max min_per_class_accuracy,0.1762295,0.8417710,294.0
max mean_per_class_accuracy,0.1326479,0.8440911,316.0


Gains/Lift Table: Avg response rate: 23.93 %, avg score: 23.93 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100119,0.9626601,4.1791734,4.1791734,1.0,0.9634913,1.0,0.9634913,0.0418414,0.0418414,317.9173441,317.9173441
,2,0.0200033,0.9613690,4.1791734,4.1791734,1.0,0.9620335,1.0,0.9627632,0.0417558,0.0835972,317.9173441,317.9173441
,3,0.0300152,0.9597695,4.1620807,4.1734720,0.9959100,0.9606213,0.9986357,0.9620487,0.0416702,0.1252674,316.2080707,317.3471976
,4,0.0400066,0.9572821,4.1534818,4.1684795,0.9938525,0.9586327,0.9974411,0.9611956,0.0414991,0.1667665,315.3481801,316.8479547
,5,0.0500184,0.9517912,4.1449880,4.1637774,0.9918200,0.9547677,0.9963160,0.9599089,0.0414991,0.2082656,314.4987973,316.3777386
,6,0.1000164,0.8603603,3.6178430,3.8908661,0.8656839,0.9072108,0.9310133,0.9335653,0.1808847,0.3891503,261.7843019,289.0866081
,7,0.1500143,0.7136729,2.9298710,3.5705781,0.7010647,0.7945520,0.8543742,0.8872338,0.1464876,0.5356379,192.9870979,257.0578100
,8,0.2000123,0.4931422,2.3839429,3.2739497,0.5704341,0.6072670,0.7833965,0.8172493,0.1191923,0.6548302,138.3942917,227.3949671
,9,0.3000082,0.2111654,1.6086908,2.7189013,0.3849304,0.3322760,0.6505835,0.6556025,0.1608625,0.8156926,60.8690841,171.8901277
,10,0.4000041,0.0977196,0.9908851,2.2869193,0.2371007,0.1441487,0.5472181,0.5277456,0.0990845,0.9147771,-0.9114897,128.6919346



<bound method ModelBase.coef_norm of >


In [207]:
bm=stackedensemble_df(aml_leaderboard_df)

In [208]:
bm

['GBM_2_AutoML_20190415_164309',
 'GLM_grid_1_AutoML_20190415_164309_model_1',
 'DRF_1_AutoML_20190415_164309']

In [209]:

aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190415_164309,0.928763,0.277231,0.167004,0.296432,0.087872
1,GBM_2_AutoML_20190415_164528,0.928615,0.277539,0.173465,0.296479,0.087900
2,StackedEnsemble_AllModels_AutoML_20190415_164309,0.928563,0.294461,0.176996,0.300726,0.090436
3,GBM_1_AutoML_20190415_164528,0.928524,0.277594,0.167987,0.296648,0.088000
4,GBM_1_AutoML_20190415_164309,0.928404,0.278311,0.167706,0.296534,0.087932
5,GBM_3_AutoML_20190415_164528,0.928247,0.278928,0.168976,0.296887,0.088142
6,StackedEnsemble_BestOfFamily_AutoML_20190415_164309,0.927830,0.295549,0.169280,0.301231,0.090740
7,GBM_3_AutoML_20190415_164309,0.925747,0.288488,0.173977,0.299740,0.089844
8,GBM_4_AutoML_20190415_164528,0.919004,0.329096,0.173070,0.315762,0.099706
9,GBM_4_AutoML_20190415_164309,0.918170,0.363044,0.180648,0.330738,0.109387


In [210]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [211]:

bm

['GBM_2_AutoML_20190415_164309',
 'DRF_1_AutoML_20190415_164309',
 'GLM_grid_1_AutoML_20190415_164309_model_1',
 'StackedEnsemble_BestOfFamily_AutoML_20190415_164309']

In [212]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]

In [213]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190415_164309,0.928763,0.277231,0.167004,0.296432,0.087872
6,StackedEnsemble_BestOfFamily_AutoML_20190415_164309,0.927830,0.295549,0.169280,0.301231,0.090740
10,DRF_1_AutoML_20190415_164309,0.917351,0.306874,0.176972,0.306518,0.093953
12,GLM_grid_1_AutoML_20190415_164309_model_1,0.906797,0.317975,0.197513,0.318892,0.101692


In [214]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [215]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'StackedEnsemble_BestOfFamily_AutoML_20190415_164309',
   'type': 'Key<Model>',
   'URL': '/3/Models/StackedEnsemble_BestOfFamily_AutoML_20190415_164309'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_adult2.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_adult2.hex'}},
 'response_column': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ColSpecifierV3',
    'schema_type': 'VecSpecifier'},
   'column_name': 'income',
   'is_member_of_frames': None}},
 'validation_frame': {'default': None, 'actual': None},
 'blending_frame': {'default': None, 'actual': None},
 'base_models': {'default': [],
  'actual': [{'__meta': {'schema_version': 3,
     'schema_nam

In [216]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

GBM_2_AutoML_20190415_164309


In [217]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

GBM_2_AutoML_20190415_164309
gbm


In [218]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [219]:
meta_data['models']=bm

In [220]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass

In [221]:
print(models_path)

C:\Users\pkash\Y\e\Q\R\models


In [222]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_2_AutoML_20190415_164309', 'varimp': [('relationship', 6503.62451171875, 1.0, 0.27605191656951894), ('capital-gain', 5131.92919921875, 0.7890875603244976, 0.2178291333687434), ('marital-status', 2236.791015625, 0.34392991348048635, 0.09494251178187708), ('occupation', 2078.765869140625, 0.3196319014720082, 0.0882349989981075), ('education', 1859.2523193359375, 0.2858794070884302, 0.07891755823451886), ('capital-loss', 1386.0009765625, 0.21311208450996713, 0.05882999937310171), ('educational-num', 1330.5340576171875, 0.20458346806764827, 0.05647565845851328), ('age', 1097.14013671875, 0.1686967220727204, 0.04656905344716994), ('hours-per-week', 654.80029296875, 0.10068236439371285, 0.02779355965563512), ('native-country', 564.0496826171875, 0.08672851293933678, 0.02394157221812824), ('workclass', 403.78973388671875, 0.06208687681140542, 0.017139201337604087), ('fnlwgt', 172.98529052734375, 0.0265982899559538, 0.00734250891981283), ('gender', 100.45161437

In [223]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_2_AutoML_20190415_164309', 'varimp': [('relationship', 6503.62451171875, 1.0, 0.27605191656951894), ('capital-gain', 5131.92919921875, 0.7890875603244976, 0.2178291333687434), ('marital-status', 2236.791015625, 0.34392991348048635, 0.09494251178187708), ('occupation', 2078.765869140625, 0.3196319014720082, 0.0882349989981075), ('education', 1859.2523193359375, 0.2858794070884302, 0.07891755823451886), ('capital-loss', 1386.0009765625, 0.21311208450996713, 0.05882999937310171), ('educational-num', 1330.5340576171875, 0.20458346806764827, 0.05647565845851328), ('age', 1097.14013671875, 0.1686967220727204, 0.04656905344716994), ('hours-per-week', 654.80029296875, 0.10068236439371285, 0.02779355965563512), ('native-country', 564.0496826171875, 0.08672851293933678, 0.02394157221812824), ('workclass', 403.78973388671875, 0.06208687681140542, 0.017139201337604087), ('fnlwgt', 172.98529052734375, 0.0265982899559538, 0.00734250891981283), ('gender', 100.45161437

In [224]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass

{'algo': 'drf', 'model_id': 'DRF_1_AutoML_20190415_164309', 'varimp': [('relationship', 41057.48828125, 1.0, 0.18630685618351636), ('capital-gain', 36127.66015625, 0.8799286480646372, 0.16393674008673434), ('occupation', 20945.40625, 0.5101482610558348, 0.09504411870480035), ('age', 19177.638671875, 0.467092349646556, 0.08702250721002164), ('marital-status', 19159.642578125, 0.4666540351148261, 0.0869408462075955), ('education', 16103.2353515625, 0.39221189667650647, 0.07307176542757407), ('educational-num', 15915.62109375, 0.38764234637846307, 0.07222042687737315), ('hours-per-week', 13230.314453125, 0.32223876829715803, 0.06003529186189207), ('fnlwgt', 11322.0244140625, 0.27576027877070614, 0.05137603059806033), ('capital-loss', 9269.908203125, 0.2257787456364775, 0.04206412829258994), ('workclass', 7137.8994140625, 0.1738513414451174, 0.032389696867926886), ('native-country', 6116.5029296875, 0.14897411375456118, 0.027754898786337835), ('race', 2452.44189453125, 0.059731902685611266

In [225]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass

In [226]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass

{'algo': 'glm', 'model_id': 'GLM_grid_1_AutoML_20190415_164309_model_1', 'coef': {'Intercept': -8.130286272637353, 'native-country.?': 0.065729266621754, 'native-country.Cambodia': 0.5572126666010029, 'native-country.Canada': 0.5901026055722867, 'native-country.China': -0.38607416793358634, 'native-country.Columbia': -1.0542652388600449, 'native-country.Cuba': 0.23552384656444758, 'native-country.Dominican-Republic': -0.525465533873249, 'native-country.Ecuador': -0.13221664295236543, 'native-country.El-Salvador': -0.29393625283317865, 'native-country.England': 0.49511135261503186, 'native-country.France': 0.5537852277774868, 'native-country.Germany': 0.21203688770410686, 'native-country.Greece': -0.004900891142730397, 'native-country.Guatemala': -0.18203537829312894, 'native-country.Haiti': 0.21250593926755865, 'native-country.Holand-Netherlands': -0.005788764323843744, 'native-country.Honduras': 0.021601287801425277, 'native-country.Hong': -0.1714622937455509, 'native-country.Hungary'

In [227]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.8])

In [228]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions
predictions_df=predictions_test(mod_best,test,run_id)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [229]:

predictions_df.describe()

Rows:9620
Cols:3




,predict,<=50K,>50K
type,enum,real,real
mins,,0.004086472610442038,0.0011299782772838634
mean,,0.7612328975641295,0.23876710243587052
maxs,,0.9988700217227161,0.995913527389558
sigma,,0.3085246308514548,0.3085246308514548
zeros,,0,0
missing,0,0,0
0,>50K,0.5260915641787428,0.4739084358212572
1,>50K,0.01171574876195558,0.9882842512380444
2,<=50K,0.9945128816183055,0.005487118381694518


In [230]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [231]:
meta_data

{'start_time': 1555360951.0231917,
 'target': None,
 'server_path': 'C:\\Users\\pkash\\Y\\e\\Q',
 'data_path': 'C:/Users/pkash/Downloads/adult.csv',
 'test_path': None,
 'max_models': 20,
 'run_time': 100,
 'run_id': 'R',
 'scale': False,
 'classification': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1555361331.7373235,
 'execution_time': 380.7141318321228,
 'run_path': 'C:\\Users\\pkash\\Y\\e\\Q\\R',
 'nthreads': 1,
 'min_mem_size': 0,
 'analysis': 0,
 'x': ['age',
  'workclass',
  'fnlwgt',
  'education',
  'educational-num',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'gender',
  'capital-gain',
  'capital-loss',
  'hours-per-week',
  'native-country'],
 'variables': {'age': 'int',
  'workclass': 'enum',
  'fnlwgt': 'int',
  'education': 'enum',
  'educational-num': 'int',
  'marital-status': 'enum',
  'occupation': 'enum',
  'relationship': 'enum',
  'race': 'enum',
  'gender': 'enum',
  'capital-gai

In [232]:
# Save logs
h2o.download_all_logs(dirname=logs_path, filename=logfile)

Writing H2O logs to C:\Users\pkash\Y\e\Q\R\logs\R_autoh2o_log.zip


'C:\\Users\\pkash\\Y\\e\\Q\\R\\logs\\R_autoh2o_log.zip'

In [233]:
# Clean up
os.chdir(server_path)

In [234]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    print(stats_test)   
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
      print(cf)   
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      print(cf_df)    
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

In [235]:
h2o.cluster().shutdown()

H2O session _sid_b830 closed.
